Video Comparison of LSTM vs Real Data

In [11]:
import os
import sys
sys.path.append(os.getenv("PROJECT_PATH"))

from utils.drawpitch import draw_pitch, draw_points_on_pitch
from utils.pitchconfig import SoccerPitchConfiguration

import pandas as pd
import numpy as np
import supervision as sv

tfdf = pd.read_csv("../../data/predicted/tf_pred_xy_comb.csv")
traindf = pd.read_csv("../../data/predicted/tf_pred_xy_comb.csv")

In [12]:
#Supervision - virtualization
CONFIG = SoccerPitchConfiguration()

edge_annotator = sv.EdgeAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    thickness=2, edges=CONFIG.edges)
vertex_annotator = sv.VertexAnnotator(
    color=sv.Color.from_hex('#FF1493'),
    radius=8)
vertex_annotator_2 = sv.VertexAnnotator(
    color=sv.Color.from_hex('#00BFFF'),
    radius=8)

color_team0 = sv.Color(255, 0, 0)  #RED
color_team1 = sv.Color(0, 0, 255)      # Blue
color_ball = sv.Color(255, 255, 255)   # White
color_gk_team0 = sv.Color(255,0,255) # yellow for team 0 goalkeeper
color_gk_team1 = sv.Color(255, 165, 0)  # Orange for team 1 goalkeeper

# === Prepare output directory ===
output_dir = "../../data/frames/frames_tf+train"
os.makedirs(output_dir, exist_ok=True)

# === Pitch configuration ===
pitch_config = SoccerPitchConfiguration()
scale = 0.1
padding = 50

In [13]:
import pandas as pd
import numpy as np
import cv2
import os
from tqdm import tqdm
import supervision as sv

#data preparation

#tf
tf_cols = [col for col in tfdf.columns if col.startswith("pred_")]
if 'frame_index' in tfdf.columns:
    tf_cols = ['frame_index'] + tf_cols
tfdf_pred = tfdf[tf_cols]

tf_team0_cols = [col for col in tf_cols if "_team_0" in col]
tf_team1_cols = [col for col in tf_cols if "_team_1" in col]
tf_ball_cols = [col for col in tf_cols if "ball" in col]

#train
train_cols = [col for col in traindf.columns if col.startswith("true_")]
if 'frame_index' in traindf.columns:
    train_cols = ['frame_index'] + train_cols
traindf_pred = traindf[train_cols]

train_team0_cols = [col for col in train_cols if "_team_0" in col]
train_team1_cols = [col for col in train_cols if "_team_1" in col]
train_ball_cols = [col for col in train_cols if "ball" in col]

#render frames
for idx, row in tqdm(tfdf_pred.iterrows(), total=len(tfdf_pred), desc="Rendering frames"):
    frame_index = int(row["frame_index"])

    # tf predictions
    tf_team0_points, tf_team1_points, tf_ball_points = [], [], []
    for col in tf_team0_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val): continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            tf_team0_points.append([x, y])
        except: continue
    for col in tf_team1_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val): continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            tf_team1_points.append([x, y])
        except: continue
    for col in tf_ball_cols:
        val = row[col]
        if not isinstance(val, str) or pd.isna(val): continue
        try:
            x_str, y_str = val.split(",")
            x, y = float(x_str), float(y_str)
            tf_ball_points.append([x, y])
        except: continue

    # Train (ground truth) data for the same frame
    train_row = traindf_pred.iloc[idx] if idx < len(traindf_pred) else None
    train_team0_points, train_team1_points, train_ball_points = [], [], []
    if train_row is not None:
        for col in train_team0_cols:
            val = train_row[col]
            if not isinstance(val, str) or pd.isna(val): continue
            try:
                x_str, y_str = val.split(",")
                x, y = float(x_str), float(y_str)
                train_team0_points.append([x, y])
            except: continue
        for col in train_team1_cols:
            val = train_row[col]
            if not isinstance(val, str) or pd.isna(val): continue
            try:
                x_str, y_str = val.split(",")
                x, y = float(x_str), float(y_str)
                train_team1_points.append([x, y])
            except: continue
        for col in train_ball_cols:
            val = train_row[col]
            if not isinstance(val, str) or pd.isna(val): continue
            try:
                x_str, y_str = val.split(",")
                x, y = float(x_str), float(y_str)
                train_ball_points.append([x, y])
            except: continue

    if not (tf_team0_points or tf_team1_points or tf_ball_points or train_team0_points or train_team1_points or train_ball_points):
        continue

    pitch = draw_pitch(
        config=pitch_config,
        background_color=sv.Color(34, 139, 34),
        line_color=sv.Color.WHITE,
        scale=scale,
        padding=padding
    )

    # Draw tf predictions
    if tf_team0_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(tf_team0_points),
            pitch=pitch,
            labels=[f"{i+1}" for i in range(len(tf_team0_points))],
            face_color=color_team0,
            #edge_color=sv.Color.BLACK,
            radius=15,
            thickness=2,
            scale=scale,
            padding=padding,
            
        )
    if tf_team1_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(tf_team1_points),
            pitch=pitch,
            labels=[f"{i+1}" for i in range(len(tf_team1_points))],
            face_color=color_team1,
            #edge_color=sv.Color.BLACK,
            radius=15,
            thickness=2,
            scale=scale,
            padding=padding
        )
    if tf_ball_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(tf_ball_points),
            pitch=pitch,
            face_color=color_ball,
            edge_color=sv.Color.BLACK,
            radius=7,
            thickness=2,
            scale=scale,
            padding=padding
        )

    # Draw train (ground truth) data with different colors and smaller radius
    if train_team0_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(train_team0_points),
            pitch=pitch,
            labels=[f"{i+1}" for i in range(len(train_team0_points))],
            face_color=sv.Color(255, 168, 168),  
            edge_color=sv.Color.BLACK,
            radius=15,
            thickness=2,
            scale=scale,
            padding=padding
        )
    if train_team1_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(train_team1_points),
            pitch=pitch,
            labels=[f"{i+1}" for i in range(len(train_team0_points))],
            face_color=sv.Color(168, 168, 255),  
            edge_color=sv.Color.BLACK,
            radius=15,
            thickness=2,
            scale=scale,
            padding=padding
        )
    if train_ball_points:
        pitch = draw_points_on_pitch(
            config=pitch_config,
            xy=np.array(train_ball_points),
            pitch=pitch,
            face_color=sv.Color(0, 0, 0),  
            edge_color=sv.Color.BLACK,
            radius=7,
            thickness=2,
            scale=scale,
            padding=padding
        )

    cv2.imwrite(os.path.join(output_dir, f"frame_{frame_index:05d}.png"), pitch)

Rendering frames: 100%|██████████| 117/117 [00:03<00:00, 34.97it/s]


In [14]:
import imageio

frame_folder = "../../data/frames/frames_tf+train"
frame_files = sorted([f for f in os.listdir(frame_folder) if f.endswith(".png")])

images = []
for filename in frame_files:
    filepath = os.path.join(frame_folder, filename)
    images.append(imageio.imread(filepath))

imageio.mimsave("../../data/animations/tf+train_frames.gif", images, fps=6.06)